In [ ]:
library(data.table)
library(dplyr)
library(taxizedb)
library(tidyr)



In [ ]:
kdf <- fread(snakemake@input$kaiju,header = F ,sep="\t", fill = T) %>%
    filter(V1=="C") %>%
    select(contig = V2, taxid = V3)
kdf

In [ ]:
l_df <- fread(snakemake@input$linf) %>%
    rowwise() %>%
    mutate(contig = strsplit(V1, split = " ")[[1]][[1]]) %>%
    select(contig, length = V2)
head(l_df)

In [ ]:
library(taxizedb)

# Download and set up the local database (e.g., NCBI)
# This step downloads the database and sets it up locally, which can take some time initially
db <- db_download_ncbi()

# Example: Fetch taxonomy information for multiple IDs


get_lineages <- function(xx){
    standard_ranks <- c("kingdom", "phylum", "class", "order", "family", "genus", "species")
    tax_info <- classification(ids, db="ncbi")
    # Print the fetched data
    #tax_info
    tt <- lapply(xx, function(x){
        tax_info <- classification(x, db="ncbi")
        df <- tax_info[[1]]
        if (!length(nrow(df))==0) {
            df$query <- x
            return(df)
        } else {
            
        return(NA) 
        }
        
    })
    tt <- tt[!is.na(tt)]
    dfo <- bind_rows(tt) %>%
        filter(rank %in% standard_ranks) %>%
        select(-id) %>%
    pivot_wider(names_from = rank, values_from = name)
    return(dfo)
}


In [ ]:
ids <- unlist(unique(unlist(kdf$taxid)))
txinf <- get_lineages(ids)

In [ ]:
txinf$query <- as.integer(txinf$query)

In [ ]:
kdf$taxid <- as.integer(kdf$taxid)

In [ ]:
kdf2 <- kdf %>%
    left_join(txinf, by=c("taxid"="query")) %>%
    left_join(l_df, by = "contig")
kdf2


In [ ]:
kdf3 <- kdf2 %>%
    rowwise() %>%
    mutate(bin = strsplit(contig, split="_")[[1]][[1]]) %>%
    filter(!is.na(species)) %>%
    group_by(bin,species, taxid) %>%
    summarise(N=n()) %>%
    arrange(-N) %>%
    ungroup() %>%
    group_by(bin) %>%
    slice_max(order_by = N,with_ties = F, n=1) %>%
    ungroup() %>%
    left_join(txinf, by=c("taxid"="query")) %>%
    rowwise() %>%
    mutate(binid = as.integer(strsplit(bin, split=".",fixed = T)[[1]][[2]])) %>%
    arrange(binid) %>%
    select(-species.x) %>%
    rename(species = species.y)


    
fwrite(x = kdf3, file = snakemake@output[[1]])